In [104]:
#サーバーコントロールファイル
server_config_path = 'C:/temp_folder/server_config.txt'
with open(server_config_path, 'r', encoding='utf-8') as file:
    file_content = file.read()
exec(file_content)

print('Server Message:', server_message)
print('Server Names:', server_names)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/temp_folder/server_config.txt'

## 0.設定

In [21]:
#0.1.ライブラリの導入
import pandas as pd
import os,glob
import requests
import time,random
from datetime import datetime, timedelta
import traceback

#全ての列を表示する
pd.set_option('display.max_columns', None)

# 忽略 InsecureRequestWarning 警告
import requests
from urllib3.exceptions import InsecureRequestWarning
import urllib3
urllib3.disable_warnings(InsecureRequestWarning)

#PDFテキスト抽出関連
import pdfplumber

#PDF画像変換関連
from pdf2image import convert_from_path
def convert_to_pic_2000(input_pdf_path, output_pic_dirpath):
    #https://pdf2image.readthedocs.io/en/latest/reference.html
    #popplerの保存フォルダをpoppler_pathで指定する
    # make dirs
    if not os.path.exists(output_pic_dirpath):
        os.makedirs(output_pic_dirpath)
    #open pdf-file
    images = convert_from_path(input_pdf_path, poppler_path = r"/opt/homebrew/bin",size=2000)
    #save every pages
    for image in images:
        output_pic_filepath = os.path.join(output_pic_dirpath, str(images.index(image)+1)+'_original.jpg')
        image.save(output_pic_filepath, 'JPEG')
        
def convert_to_pic_500(input_pdf_path, output_pic_dirpath):
    #https://pdf2image.readthedocs.io/en/latest/reference.html
    #popplerの保存フォルダをpoppler_pathで指定する
    # make dirs
    if not os.path.exists(output_pic_dirpath):
        os.makedirs(output_pic_dirpath)
    #open pdf-file
    images = convert_from_path(input_pdf_path, poppler_path = r"/opt/homebrew/bin", size=500)
    #save every pages
    for image in images:
        output_pic_filepath = os.path.join(output_pic_dirpath, str(images.index(image)+1)+'.jpg')
        image.save(output_pic_filepath, 'JPEG')

# =======================
# 2. 追加：指定ページだけを画像化する関数（新規）
# =======================
# def convert_to_pic_500_for_page(input_pdf_path, output_pic_dirpath, page_number):
#     """
#     指定したPDFファイルの特定のpage_number(1始まり)のみを
#     解像度size=500で画像化し、JPEGファイルとして保存する。
#     """
#     if not os.path.exists(output_pic_dirpath):
#         os.makedirs(output_pic_dirpath)
#     # pdf2imageの first_page, last_page を使って特定ページのみ画像化
#     images = convert_from_path(
#         input_pdf_path,
#         poppler_path=r"/opt/homebrew/bin",
#         size=500,
#         first_page=page_number,
#         last_page=page_number
#     )
#     # 原則1枚だけ返る想定だが、念のためループ
#     for idx, image in enumerate(images):
#         output_pic_filepath = os.path.join(output_pic_dirpath, f"p{page_number}_{idx+1}.jpg")
        # image.save(output_pic_filepath, 'JPEG')


from pdf2image import convert_from_path

POPPLER_PATH = "/opt/homebrew/bin"

def get_page_image_in_memory(pdf_path, page_number, poppler_path=POPPLER_PATH, size=1000):
    """
    指定したPDFファイルの特定のpage_number(1始まり)を
    解像度size=500で画像化し、PIL.Imageオブジェクトのリストとして返す。
    """
    images = convert_from_path(
        pdf_path,
        poppler_path=poppler_path,
        size=size,
        first_page=page_number,
        last_page=page_number
    )
    return images  # [PIL.Image, ...]


#GCPへのアップロード関連
#!pip install --upgrade google-cloud-storage
from google.cloud import storage
bucket_name_prod='gf-p'
service_account_path_prod='../config/gcp_auth/g-finder-prod-9a658eb38e6c.json'
bucket_name_backup='backup-prod-cs'
service_account_path_backup='../config/gcp_auth/glocal-backup-392107-0320e3f6fb57.json'

#0.2,取り込みファイルのパス指定
#0.2.1.Teams保管ファイル　親パス
path_teams='/Users/takenaka/Library/CloudStorage/OneDrive-共有ライブラリ-プライマルホールディングス株式会社/G-Finder - G-Finder関係資料 - G-Finder関係資料/00_取り込み用/'
#0.2.1.1.書類提出フォルダ（本番）
path_teams_inputdata_folder="00_文書インプットデータ/開発_竹中/"
#0.2.1.2.省庁・自治体の団体コード付与ルールを記載する資料
path_teams_code='01_基礎データ/省庁・自治体基礎データ.xlsx'
#0.2.1.3.カテゴリーIDの付与ルール を記載する資料
path_teams_category="01_基礎データ/インデックスカテゴリールール.xlsx"
#0.2.1.4.解析結果レポートのフォルダ（本番）
path_teams_report_folder="02_解析結果レポート/本番_解析結果/"

#0.2.2.OneDrive保管ファイル　親パス
#path_onedrive="C:/Users/Administrator/OneDrive - プライマルホールディングス株式会社/CloudDesktop/ElasticDataUpload/"
#0.2.2.1. CSVなどのファイルを保存するローカルフォルダ
local_save_folder='/Users/takenaka/Library/CloudStorage/OneDrive-プライマルホールディングス株式会社/PassageDrive/Workspace/Documents/gfinder-data-tools/config/bunsyo/'
#0.2.2.2. 前回までの解析場所を記載する資料
last_submit_id_folder='/Users/takenaka/Library/CloudStorage/OneDrive-プライマルホールディングス株式会社/PassageDrive/Workspace/Documents/gfinder-data-tools/config/last_submit/'

#0.2.3. GCPアップ予定のPDFと画像などを一時保存するための臨時フォルダー
temp_folder='/Users/takenaka/Library/CloudStorage/OneDrive-プライマルホールディングス株式会社/PassageDrive/Workspace/Documents/gfinder-data-tools/config/temp_folder/'

#0.3.取り込み
#0.3.1.Configファイルをローディング（省庁・自治体基礎データ）
def format_code(row):
    code=row['code']
    if code=="":
        pass
    elif row['affiliation_name']=='省庁':
        return str(int(code)).zfill(5)
    else:
        return str(int(code)).zfill(6)
df_config_code1=pd.read_excel(path_teams+path_teams_code,sheet_name="自治体基礎データ").fillna("")
df_config_code1=df_config_code1[["団体コード","都道府県名","市区町村名"]].rename(columns={"団体コード":"code","都道府県名":"affiliation_name","市区町村名":"organization_name"})
df_config_code2=pd.read_excel(path_teams+path_teams_code,sheet_name="省庁基礎データ").fillna("")
df_config_code2=df_config_code2[["団体コード","機関名称"]].rename(columns={"団体コード":"code","機関名称":"organization_name"})
df_config_code2["affiliation_name"]="省庁"
#...df_config_code=df_config_code1.append(df_config_code2)
df_config_code = pd.concat([df_config_code1, df_config_code2], ignore_index=True)
df_config_code['code'] = df_config_code.apply(format_code, axis=1)

#0.3.2.Configファイルをローディング（カテゴリーID基礎データ）
df_config_category=pd.read_excel(path_teams+path_teams_category,sheet_name="category").fillna("")
df_config_subcategory=pd.read_excel(path_teams+path_teams_category,sheet_name="sub_category").fillna("")

#0.3.3.前回までの解析場所を記載する資料をローディング
# last_submit_id_folder='../config/last_submit/'
# last_submit_id_files = glob.glob(last_submit_id_folder + "*.csv") 
# last_submit_id_tables=pd.concat((pd.read_csv(file) for file in last_submit_id_files), ignore_index=True)
# #0.3.3.1.過去解析結果から、前回submit_idを獲得
# last_submit_id_table=last_submit_id_tables[last_submit_id_tables['server_name'].isin(server_names)]
#0.3.3.2.辞書化　サンプル{'Mercury': 0, 'Venus': 0}
# last_submit_id_dict =last_submit_id_table.set_index('server_name')['submit_id'].to_dict()
# for server_name in server_names:
#     if server_name not in last_submit_id_dict:
#         last_submit_id_dict[server_name] = 0

#0.4.mecab関連
import sys,MeCab,collections,math,re

'''
mecabrc: (デフォルト)
-Ochasen: (ChaSen 互換形式)
-Owakati: (分かち書きのみを出力)
-Oyomi: (読みのみを出力)

WindowsでPythonでMeCab（mecab-ipadic-NEologd） - Qiita
https://qiita.com/yakipudding/items/0372dc79bb5722fa4b8b
"-d ..\dic\ipadic-neologd" neologd搭載
'''

def calculate_digit_ratio(word: str) -> float:
    #ワードの中の数字の％を算出
    digit_chars = len(re.findall(r"\d", word))
    total_chars = len(word)
    ratio = digit_chars / total_chars if total_chars > 0 else 0
    return ratio

def mecab_list(text):
    #https://qiita.com/menon/items/2b5ad487a98882289567
    tagger = MeCab.Tagger("-r /opt/homebrew/etc/mecabrc -d /opt/homebrew/lib/mecab/dic/ipadic")
    tagger.parse('')
    node = tagger.parseToNode(text)
    word_class = []
    while node:
        word = node.surface
        wclass = node.feature.split(',')
        if wclass[1] in ["一般","固有名詞"]: #フィルタリング2列目
        #if wclass[1] in ["形容動詞語幹","サ変接続","一般","固有名詞","ナイ形容詞語幹"]:
        #if wclass[0] != u'BOS/EOS':
            if wclass[0] in ["副詞","記号"]: #フィルタリング1列目
                pass
            elif wclass[6] == "*": #辞書型がないとき
                if len(word)<=1: #長さ＜１の単語を捨てる
                    pass
                elif word[0]==word[1]: #目次記号「・・・」などを捨てる
                    pass
                elif calculate_digit_ratio(word)>0.2: #「4月」など数字を含む無意味な頻出語を捨てる
                    pass
                else:
                    #word_class.append((word,wclass[0],wclass[1],wclass[5],""))
                    word_class.append(word) #単語のみアウトプット
            else: #辞書型があるとき
                if len(wclass[6])<=1: #長さ＜１の単語を捨てる
                    pass
                elif calculate_digit_ratio(wclass[6])>0.2: #「4月」など数字を含む無意味な頻出語を捨てる
                    pass
                else:
                    #word_class.append((word,wclass[0],wclass[1],wclass[4],wclass[5],wclass[6]))
                    word_class.append(wclass[6])
        node = node.next
    return word_class

#0.5. Elestic API
from elasticsearch import Elasticsearch
elastic_password_path='../config/Elastic_auth/password'

#0.6 チェック機構
# gfinder-data-tools のパスを追加
current_dir = os.getcwd()  # Notebook起動時のディレクトリ
parent_dir = os.path.abspath(os.path.join(current_dir, "../config/gfinder-data-tools"))
sys.path.append(parent_dir)
import DataValidator.LocalGovDataValidator.LocalGovDataValidator as LGDValidator
import importlib
importlib.reload(LGDValidator)

#0.7 twilio
# from twilio.rest import Client
# twilio_password_path='../config/twilio_auth/password'

# with open(twilio_password_path, 'r') as file:
#     text_content = file.read()

# twilio_variables = {}
# for line in text_content.splitlines():
#     line = line.strip()
#     if line:  # 确保行不为空
#         exec(line, twilio_variables)

# def send_twilio_message(variables, message_body):
#     try:
#         account_sid = variables['account_sid']
#         auth_token = variables['auth_token']
#         from_number = variables['from_']
#         to_number = variables['to']
        
#         client = Client(account_sid, auth_token)
#         message = client.messages.create(
#           from_=from_number,
#           body=message_body,
#           to=to_number
#         )
#         return message.sid
#     except Exception as e:
#         print(f"TwillioError: {e}")
#         return None

#0.8 ハッシュ値
import hashlib
def calculate_sha256(file_path):
    sha256_hash = hashlib.sha256()
    with open(file_path, "rb") as f:
        for byte_block in iter(lambda: f.read(4096), b""):
            sha256_hash.update(byte_block)
    return sha256_hash.hexdigest()

# Gemini OCR用
import google.generativeai as genai
import PIL.Image
from dotenv import load_dotenv, find_dotenv
from google.generativeai.types.content_types import to_contents

# =======================
# 3. Gemini初期設定
# =======================
_ = load_dotenv(find_dotenv())
genai.configure()  # .env内のGOOGLE_API_KEYを読み込む想定
model = genai.GenerativeModel('gemini-2.0-flash')

def gemini_ocr_image(pil_image, prompt="画像から必ず全てのテキストを抽出して、抽出したテキストは必ず***と***で挟んでください。"):
    """
    PIL.ImageオブジェクトをGeminiへ渡し、OCR実行して返却されたテキストを返す。
    """
    response = model.generate_content([pil_image, prompt])
    return response.text

import re


def extract_inside_asterisks(text):
    """
    OCR結果のテキストから、"*** ... ***" で囲まれた部分をすべて抽出し、半角スペースで連結して返す。
    もしマッチしなければ、空文字列を返す。
    """
    matches = re.findall(r'\*\*\*(.*?)\*\*\*', text, flags=re.DOTALL)
    extracted = ' '.join(match.strip() for match in matches if match.strip())
    return extracted

import time
import traceback

def gemini_ocr_with_backoff(pil_img, prompt, max_retries=5, initial_delay=5):
    """
    Gemini OCR API へのリクエストに対して、指数バックオフを実装。
    
    Args:
        pil_img: OCR対象のPIL.Imageオブジェクト
        prompt: OCRに対するプロンプト文字列
        max_retries: 最大リトライ回数（デフォルト: 5）
        initial_delay: 初回待機秒数（デフォルト: 5秒）
        
    Returns:
        gemini_ocr_image の結果テキスト（成功した場合）
        
    Raises:
        最終的な例外をそのまま再送出します。
    """
    delay = initial_delay
    attempt = 0
    while attempt < max_retries:
        try:
            # OCRリクエスト実行
            ocr_text = gemini_ocr_image(pil_img, prompt=prompt)
            return ocr_text
        except Exception as e:
            attempt += 1
            print(f"[OCRリトライ] 試行{attempt}/{max_retries}でエラー: {repr(e)}")
            print(traceback.format_exc())
            if attempt >= max_retries:
                raise
            # 待機してからリトライ
            time.sleep(delay)
            delay *= 2  # 待機時間を倍にする


## 1-3.解析対象ファイル選定、データ整形

In [22]:
#1.INPUTDATAをローディング
#1.1.対象フォルダ内のExcelを読み取って合体する
inpudata_files = glob.glob(path_teams+path_teams_inputdata_folder + "*竹中.xlsx") 
inputdata=pd.concat((pd.read_excel(file) for file in inpudata_files), ignore_index=True)

#1.2.file_id列に値が存在する行のみを抽出する
inputdata=inputdata[~inputdata.file_id.isna()].fillna("")

#1.3.該当サーバーの解析対象行を取得
print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'),' INPUTDATA Excelローディング完了')
# print('■現在のサーバー：'+",".join(server_names))
print('■Excelローディング：')
# print(inputdata.groupby('server_name')['file_id'].count())
# inputdata=inputdata[inputdata['server_name'].isin(server_names)]

# 1.4. 未解析の行を取得し、さらに submit_id が最も小さい行を取得
# def filter_next_unprocessed(inputdata, last_submit_id_dict):
#     # 各 server_name の最後に解析された submit_id より大きいすべてのレコードをフィルタリング
#     # filtered_data = inputdata[inputdata.apply(lambda row: row['submit_id'] > last_submit_id_dict.get(row['server_name'], -1), axis=1)].copy()
    
#     # # 各 server_name の次の未解析の最小 submit_id を計算
#     # min_submit_id_by_server = filtered_data.groupby('server_name')['submit_id'].transform('min')
    
#     # .loc を使用して、元の DataFrame に変更を加える
#     filtered_data.loc[:, 'min_submit_id_for_server'] = min_submit_id_by_server
    
#     # グローバル最小の submit_id を見つける
#     global_min_submit_id = min_submit_id_by_server.min()
    
#     # グローバル最小 submit_id のレコードをフィルタリング
#     updated_inputdata = filtered_data[filtered_data['submit_id'] == global_min_submit_id].drop(columns=['min_submit_id_for_server'])
    
#     # 更新後の inputdata を返す
#     return updated_inputdata

# inputdata = filter_next_unprocessed(inputdata, last_submit_id_dict)
# inputdata

2025-04-24 15:29:24  INPUTDATA Excelローディング完了
■Excelローディング：


In [23]:
#1.5.補助列を追加
inputdata[['number_of_pages',
           '重複チェック', 'URL判定', 'DL判定', 'DLエラー詳細', '画像化', '画像化エラー詳細',
           'テキスト解析', '解析エラー詳細','OCR解析', 'OCRエラー詳細',
           'GCP格納','GCPエラー詳細', '画像格納' , '画像格納エラー詳細',
           'Elastic格納' , 'Elasticエラー詳細']] = '-'

#1.6. astype()
inputdata['fiscal_year_start']=inputdata['fiscal_year_start'].astype(int)
inputdata['fiscal_year_end']=inputdata['fiscal_year_end'].astype(int)
inputdata['submit_id']=inputdata['submit_id'].astype(int)
inputdata.collected_at= pd.to_datetime(inputdata.collected_at)

#1.7 codeフォーマット変換
def format_code(row):
    if any(row['affiliation'].endswith(suffix) for suffix in ['都', '道', '府', '県']):
        # 都道府県は6桁
        return str(row['code']).zfill(6)
    else:
        # その他（省庁）は5桁
        return str(row['code']).zfill(5)
    
inputdata['code'] = inputdata['code'].astype(int)
inputdata['code'] = inputdata.apply(format_code, axis=1)

#1.8 
#数字、スペース、アルファベットは半角に統一。
#【】（）「」・＜＞は、全角に統一。
import pandas as pd

# 示例DataFrame
def half_to_full(text):
    result = ""
    for char in text:
        code = ord(char)
        # 数字、英文字母、空格的半角转全角
        if 0x21 <= code <= 0x7E:
            result += chr(code + 0xFEE0)
        elif code == 0x20:  # 半角空格
            result += chr(0x3000)
        else:
            result += char
    return result

def specific_symbols_to_full(text):
    half_to_full_map = {
        '(': '（', ')': '）',
        '[': '【', ']': '】',
        '<': '＜', '>': '＞',
    }
    for half, full in half_to_full_map.items():
        text = text.replace(half, full)
    return text

inputdata.title = inputdata.title.apply(half_to_full)
inputdata.title = inputdata.title.apply(specific_symbols_to_full)

#1.9 reset_index()
inputdata=inputdata.reset_index(drop=True)

#2. 団体コード/カテゴリーID付与
#2,1  団体コード付与
inputdata.loc[:, "affiliation_code"] = inputdata['code'].map(lambda x:str(x)[:2])
inputdata.loc[:, "organization_code"] = inputdata['code'].map(lambda x:str(x)[2:])

#2,2 カテゴリーID付与
inputdata_category=pd.merge(inputdata,df_config_category,
         left_on=['category_name'],
         right_on=['category（論理名称）'], how='left')
time.sleep(3)
inputdata.loc[:, "category"] = inputdata_category["categoryにセットする値（※データ型はint）"]
del inputdata_category

#2,3 サブカテゴリーID付与
inputdata_subcategory=pd.merge(inputdata,df_config_subcategory,
         left_on=['sub_category_name'],
         right_on=['sub_categoryにセットできる値（論理名称）'], how='left')
time.sleep(3)
inputdata.loc[:, "sub_category"] = inputdata_subcategory["sub_categoryにセットできる値（※データ型はint）"].fillna('-')
del inputdata_subcategory

#3.URL重複チェック (0＝重複なし、1＝自重複、２＝過去と重複)
#3.1.今回提出分での比較
inputdata['重複チェック'] = 0
inputdata['重複チェック'].where(~inputdata.duplicated(subset=['source_url', 'category', 'sub_category'], keep='first'), 1, inplace=True)

#3.2.過去解析分との比較
# (source_url, category, sub_category)の組み合わせをtupleとしてsetに保存
# url_set = set(tuple(x) for x in last_submit_id_tables[['source_url', 'category', 'sub_category']].to_numpy())

# inputdata内の各行をイテレートし、重複チェックが0の行だけを確認
# for index, row in inputdata.loc[inputdata['重複チェック'] == 0].iterrows():
#     # 行から(source_url, category, sub_category)の組み合わせをtupleとして抽出
#     row_tuple = tuple(row[['source_url', 'category', 'sub_category']])
    
#     # この組み合わせがurl_set内に存在するかチェック
#     if row_tuple in url_set:
#         inputdata.at[index, '重複チェック'] = 2
# del url_set

#reindex
inputdata=inputdata.reindex(columns=['title','category','category_name', 'sub_category', 'sub_category_name',
        'file_id','number_of_pages','code', 'affiliation_code','affiliation','organization_code','organization',
        'fiscal_year_start', 'fiscal_year_end', 'source_url',
        'collected_at', 'modified_at', 'submit_id', 'server_name',
        '重複チェック', 'URL判定', 'DL判定', 'DLエラー詳細', '画像化', '画像化エラー詳細',
        'テキスト解析', '解析エラー詳細','OCR解析', 'OCRエラー詳細',
        'GCP格納','GCPエラー詳細', '画像格納' , '画像格納エラー詳細', 
        'Elastic格納' , 'Elasticエラー詳細'])

#カテゴリーID付与失敗有無のチェック
print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'),' INPUTDATA構成完了')
print('カテゴリID付与失敗数：',len(inputdata[inputdata['category'].isna()].groupby(['category']).count()))
print('サブカテゴリーID付与失敗数：',len(inputdata[inputdata['sub_category'].isna()].groupby(['sub_category']).count()))

inputdata

2025-04-24 15:29:32  INPUTDATA構成完了
カテゴリID付与失敗数： 0
サブカテゴリーID付与失敗数： 0


/var/folders/pq/jj6vyqq119j3qpdg8q0j3p740000gp/T/ipykernel_22463/3192950156.py:85: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  inputdata['重複チェック'].where(~inputdata.duplicated(subset=['source_url', 'category', 'sub_category'], keep='first'), 1, inplace=True)


,title,category,category_name,sub_category,sub_category_name,file_id,number_of_pages,code,affiliation_code,affiliation,organization_code,organization,fiscal_year_start,fiscal_year_end,source_url,collected_at,modified_at,submit_id,server_name,重複チェック,URL判定,DL判定,DLエラー詳細,画像化,画像化エラー詳細,テキスト解析,解析エラー詳細,OCR解析,OCRエラー詳細,GCP格納,GCPエラー詳細,画像格納,画像格納エラー詳細,Elastic格納,Elasticエラー詳細
0,小谷村第６次総合計画,4,（自治体）予算,6,（自治体）補正予算,AA0163873,-,001010,00,長野県,1010,小谷村,2024,2024,https://www.vill.otari.nagano.jp/www/contents/...,2024-09-06 11:24:00,,202502131,Venus,0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-


## 4.ダウンロード

In [24]:
#4,1 URL判定
for index, row in inputdata[inputdata['重複チェック'] == 0].iterrows():
    if row['source_url'].startswith(("http://", "https://")):
        inputdata.at[index, 'URL判定'] = 0
    else:
        #error1.URLではありません
        inputdata.at[index, 'URL判定'] = 1
                
#4.2 ダウンロード
#4.2.1.ドメインを軸にシャフルする
from urllib.parse import urlparse
from collections import defaultdict

shuffled_data=inputdata.copy()
shuffled_data['domain'] = shuffled_data['source_url'].apply(lambda x: urlparse(x).netloc)

# 各ドメインの行を辞書に収集する
domain_rows = defaultdict(list)
for index, row in shuffled_data.iterrows():
    domain_rows[row['domain']].append(row)

# 各ドメインのリストから交互に行を取り出し、順序をシャッフルする
shuffled_data_list = []
while domain_rows:  # 辞書が空でない間処理を続ける
    for domain in list(domain_rows):  # イテレート中に辞書を変更しないように list() を使用
        shuffled_data_list.append(domain_rows[domain].pop(0))  # 各リストから1つ要素を取り出す
        if not domain_rows[domain]:  # 現在のドメインのリストが空になった場合は辞書から削除
            del domain_rows[domain]

# リストを DataFrame に変換
shuffled_data = pd.DataFrame(shuffled_data_list)

#4.2.2.シャフルのままDLする
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'}

#進捗表示関連
print(datetime.now().strftime('%Y-%m-%d %H:%M:%S')+' ファイルDL開始')
total_rows = len(shuffled_data)
readed_files = 0

#スパム対策関連
recent_downloads = {}

for index, row in shuffled_data.iterrows():
    #スパム対策：直近DLしたことのあるサイトの場合、5秒以上待つ
    domain = row['domain']
    now = datetime.now()
    
    if domain in recent_downloads:
        last_download_time = recent_downloads[domain]
        time_diff = now - last_download_time
        
        if time_diff < timedelta(seconds=5):
            time.sleep(random.randint(5, 7)) 
            
    try:
        response = requests.get(row['source_url'],headers=headers,verify=False)
        response.raise_for_status() #リクエスト結果を確認（ダウンロード不可の時は即終了）
        save_as=temp_folder+'pdf/'+row.file_id+'.pdf'

        with open(save_as, 'wb') as file:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    file.write(chunk)
                    file.flush()
        inputdata.at[index, 'DL判定'] = 0
        
        #ハッシュ値取得
        inputdata.at[index, 'hash'] = calculate_sha256(save_as)
            
    except Exception as e:
        #error1.ダウンロードに失敗しました
        inputdata.at[index, 'DL判定'] = 1
        inputdata.at[index, 'DLエラー詳細'] = str(e)
    
    if len(recent_downloads) >= 10:
        oldest_domain = min(recent_downloads, key=recent_downloads.get)
        del recent_downloads[oldest_domain]
    
    recent_downloads[domain] = datetime.now()
    
    #進捗表示
    readed_files += 1
    percentage = (readed_files / total_rows) * 100
    current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(f'\r{current_time} 進捗: {percentage:.2f}%', end='')

step_message=datetime.now().strftime('%Y-%m-%d %H:%M:%S')+' ファイルDL完了'
print('\n'+step_message)
# twillo_message=server_message+"より\n"+step_message
#send_twilio_message(twilio_variables,twillo_message)

#以下追加処理(仮：画像化)
#使うのはtemp_folderのPDFファイル

#進捗表示関連
# print(datetime.now().strftime('%Y-%m-%d %H:%M:%S')+' 画像化開始')
# total_rows = len(inputdata[inputdata['DL判定'] == 0])
# readed_files = 0

# for index, row in inputdata[inputdata['DL判定'] == 0].iterrows():   
#     file_id=row.file_id
#     pdf_path=temp_folder+'pdf/'+file_id+'.pdf'
        
#     #画像化(pdf2image)
#     preview_dirpath=temp_folder+'preview/'+file_id+'/'
#     try:
#         convert_to_pic_500(pdf_path, preview_dirpath)
#         inputdata.at[index, '画像化'] = 0
        
#     except Exception as e:
#         inputdata.at[index, '画像化'] = 1
#         inputdata.at[index, '画像化エラー詳細'] = str(e)
    
#     #進捗表示
#     readed_files += 1
#     percentage = (readed_files / total_rows) * 100
#     current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
#     print(f'\r{current_time} 進捗: {percentage:.2f}%', end='')

# step_message=datetime.now().strftime('%Y-%m-%d %H:%M:%S')+' 画像化完了'
# print('\n'+step_message)
# # twillo_message=server_message+"より\n"+step_message

2025-04-24 15:29:36 ファイルDL開始
2025-04-24 15:29:37 進捗: 100.00%
2025-04-24 15:29:37 ファイルDL完了


## 5.PDFからテキストを抽出し、CSVとして保存

In [25]:
print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 'テキスト解析開始')

# DL判定=0 の行だけ対象
target_df = inputdata[inputdata['DL判定'] == 0].copy()
total_rows = len(target_df)
readed_files = 0

for index, row in target_df.iterrows():
    readed_files += 1
    percentage = (readed_files / total_rows) * 100
    current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(f'\r{current_time} 進捗: {percentage:.2f}%', end='')

    file_id = row.file_id
    pdf_path = os.path.join(temp_folder, 'pdf', f"{file_id}.pdf")

    # ---------- 1) PDFを開いてページ数を取得 ----------
    try:
        with pdfplumber.open(pdf_path) as pdf:
            number_of_pages = len(pdf.pages)
            inputdata.at[index, 'number_of_pages'] = number_of_pages
    except Exception as e:
        inputdata.at[index, 'テキスト解析'] = 1
        inputdata.at[index, '解析エラー詳細'] = "[PDF開けません] " + str(e)
        continue

    # ページ単位のテキスト結果をためるDataFrame
    df_content = pd.DataFrame(columns=['_id','file_id','file_page','content_text','TF'])
    # OCRを必要とするページをリストで管理
    pages_for_ocr = []
    # エラーフラグ（文字化け・空白・文字数不足）
    error_check_3 = error_check_4 = error_check_5 = 0

    # ---------- 2) pdfplumberでテキスト抽出 ----------
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for i, page in enumerate(pdf.pages):
                page_num = page.page_number
                _id = f"{file_id}E{str(page_num).zfill(8)}E"

                # 画像があるかどうかチェック
                images_info = page.images
                has_image = (len(images_info) > 0)
                if has_image:
                    print(f"\n→ 画像あり: file_id={file_id}, page={page_num}")

                # pdfplumberテキスト抽出
                content_text = page.dedupe_chars(tolerance=1).extract_text() or ""
                s_text = content_text.strip()

                # OCR対象かどうか判定（画像があれば即OCR）
                if has_image:
                    pages_for_ocr.append(page_num)

                #文字化けの場合
                elif "cid:" in content_text:
                    error_check_3 = 1
                    pages_for_ocr.append(page_num)
                
                # 空白ページ
                elif s_text == "":
                    error_check_4 = 1
                    pages_for_ocr.append(page_num)
                
                else:
                    # pdfplumber で正常にテキスト取得
                    tokens = mecab_list(content_text)
                    tf_result = dict(collections.Counter(tokens).most_common(50))
                    row_data = pd.DataFrame(
                        [[_id, file_id, page_num, content_text, tf_result]],
                        columns=['_id','file_id','file_page','content_text','TF']
                    )
                    df_content = pd.concat([df_content, row_data], ignore_index=True)

        # ---------- 3) 必要ページだけ OCR -----------
        if pages_for_ocr:
            print(f"\nOCR対象: file_id={file_id}, pages={pages_for_ocr}")

            for pnum in pages_for_ocr:
                try:
                    # (A) PDFを画像化 (メモリ上)
                    image_list = get_page_image_in_memory(pdf_path, pnum, size=1500)
                    # (B) 1ページ1イメージ想定だが、念のためループ
                    from PIL import ImageEnhance  # PillowのImageEnhanceを利用
                    for pil_img in image_list:
                        # コントラスト調整（例: 1.5倍）
                        enhancer = ImageEnhance.Contrast(pil_img)
                        pil_img_adjusted = enhancer.enhance(1.5)  # 必要に応じて調整
                        # (B) GeminiでOCR（指数バックオフ付き）
                        ocr_text = gemini_ocr_with_backoff(
                            pil_img_adjusted, 
                            prompt="画像から必ず全てのテキストを抽出して、抽出したテキストは必ず***と***で挟んでください。", 
                            max_retries=5, 
                            initial_delay=5
                        )
                        # OCR結果から各ページごとに角括弧内のテキストだけを抽出（複数ある場合は連結）
                        ocr_text = extract_inside_asterisks(ocr_text)

                        # (D) DataFrameにOCR結果を格納
                        _id_ocr = f"{file_id}E{str(pnum).zfill(8)}E_OCR"
                        tokens = mecab_list(ocr_text)
                        tf_result = dict(collections.Counter(tokens).most_common(50))
                        row_data = pd.DataFrame(
                            [[_id_ocr, file_id, pnum, ocr_text, tf_result]],
                            columns=['_id','file_id','file_page','content_text','TF']
                        )
                        df_content = pd.concat([df_content, row_data], ignore_index=True)

                except Exception as e:
                    error_details = traceback.format_exc()
                    print(f"[OCRエラー] file_id={file_id}, page={pnum}, エラー詳細:\n{error_details}")
                    continue

    except Exception as e:
        import traceback
        traceback.print_exc()
        inputdata.at[index, 'テキスト解析'] = 6
        inputdata.at[index, '解析エラー詳細'] = "[その他の不明エラー] " + str(e)
        continue

    # ---------- 4) 結果をCSVに出力 ----------
    if not df_content.empty:
        df_content = df_content.sort_values(by='file_page').reset_index(drop=True)
        csv_path = os.path.join(local_save_folder, 'csv', f"{file_id}.csv")
        df_content.to_csv(csv_path, index=False, encoding='utf-8')
        inputdata.at[index, 'テキスト解析'] = 0  # 正常完了
    else:
        # df_contentが空 → ページ抽出エラー等
        if error_check_3 == 1:
            inputdata.at[index, 'テキスト解析'] = 3
            inputdata.at[index, '解析エラー詳細'] = "[文字化けです]"
        elif error_check_4 == 1:
            inputdata.at[index, 'テキスト解析'] = 4
            inputdata.at[index, '解析エラー詳細'] = "[空白ページです]"
        elif error_check_5 == 1:
            inputdata.at[index, 'テキスト解析'] = 5
            inputdata.at[index, '解析エラー詳細'] = "[文字数が足りません]"
        else:
            inputdata.at[index, 'テキスト解析'] = 2
            inputdata.at[index, '解析エラー詳細'] = "[記入できるデータがありません]"

print('\n' + datetime.now().strftime('%Y-%m-%d %H:%M:%S') + ' テキスト解析完了')


2025-04-24 15:29:41 テキスト解析開始
2025-04-24 15:29:41 進捗: 100.00%
→ 画像あり: file_id=AA0163873, page=1

→ 画像あり: file_id=AA0163873, page=2

→ 画像あり: file_id=AA0163873, page=3

→ 画像あり: file_id=AA0163873, page=4

→ 画像あり: file_id=AA0163873, page=5

→ 画像あり: file_id=AA0163873, page=6

→ 画像あり: file_id=AA0163873, page=7

→ 画像あり: file_id=AA0163873, page=8

→ 画像あり: file_id=AA0163873, page=12

→ 画像あり: file_id=AA0163873, page=14

→ 画像あり: file_id=AA0163873, page=15

→ 画像あり: file_id=AA0163873, page=16

→ 画像あり: file_id=AA0163873, page=17

→ 画像あり: file_id=AA0163873, page=18

→ 画像あり: file_id=AA0163873, page=19

→ 画像あり: file_id=AA0163873, page=20

→ 画像あり: file_id=AA0163873, page=21

→ 画像あり: file_id=AA0163873, page=22

→ 画像あり: file_id=AA0163873, page=23

→ 画像あり: file_id=AA0163873, page=24

→ 画像あり: file_id=AA0163873, page=25

→ 画像あり: file_id=AA0163873, page=26

→ 画像あり: file_id=AA0163873, page=27

→ 画像あり: file_id=AA0163873, page=28

→ 画像あり: file_id=AA0163873, page=29

→ 画像あり: file_id=AA0163873, page=30

→ 画像あり: fil

## 6.TF-IDF特徴語タグの作成

In [91]:
filtered_data = inputdata[inputdata['テキスト解析'] == 0]
total_rows = len(filtered_data)

#6.1.最も合理的な解析回数を算出
def calculate_group_distribution(total_files, min_files_per_group=300, max_files_per_group=500):
    for group_count in range(4, 0, -1):  # グループ数を4から1まで試す
        if total_files / group_count <= max_files_per_group and (total_files / group_count >= min_files_per_group or group_count == 1):
            return group_count, math.ceil(total_files / group_count)
    return 1, total_files  
num_batches, batch_size = calculate_group_distribution(total_rows)

#進捗表示関連
print(datetime.now().strftime('%Y-%m-%d %H:%M:%S')+' TFIDF解析開始（対象ファイル数'+str(batch_size)+'*'+str(num_batches)+'組）')
readed_files = 0
for n in range(num_batches):
    start_index = n * batch_size
    end_index = min((n + 1) * batch_size, total_rows)  # 合計長さを超えないようにする
    batch_data = filtered_data.iloc[start_index:end_index]
    
    # 6.2..作業用データフレームの形成
    df_contents=pd.DataFrame(columns=['_id', 'file_id', 'file_page', 'content_text', 'TF'])
    for index, row in batch_data.iterrows():
        #...df_contents=df_contents.append(pd.read_csv(local_save_folder+'csv/'+row.file_id+'.csv',encoding="utf-8"))
        df_contents = pd.concat([df_contents, pd.read_csv(local_save_folder + 'csv/' + row.file_id + '.csv', encoding="utf-8")])

    df_contents.TF=df_contents.TF.map(lambda x:eval(x))

    #6.3 IDF
    #【Python】TF-IDF を使って自分のブログの特徴を取得してみた
    #https://dev.classmethod.jp/articles/python-tfidf-blog/#orgdcd1e4a
    #全文書数J
    j=len(df_contents)

    #単語Wiが登場する文書数
    idf_doc={}
    for row_tf in df_contents.TF:
        for term,frequence in row_tf.items():
            if term in idf_doc:
                idf_doc[term]=idf_doc[term]+1
            else:
                idf_doc[term]=1

    #idf
    idf={}
    for k,v in idf_doc.items():
        idf[k]=math.log((1+j)/v)

    #tfidf score
    tfidfs=[]
    for row_tf in df_contents.TF:
        row_sum=sum(row_tf.values())
        tfidflist={}
        for term,frequence in row_tf.items():
            TermFrequence=frequence/row_sum
            tfidf=round(TermFrequence*idf[term],3)
            tfidflist[term]=tfidf

        #order by tf-idf_value, and pick up top30 as dict
        sorted_tfidf=dict(sorted(tfidflist.items(), key=lambda item:item[1],reverse=True)[:30])
        tfidfs.append(list(sorted_tfidf.keys()))

    df_contents["tags"]=tfidfs

    #6.4 タグ結果をCSVに追記
    for index, row in batch_data.iterrows():
        file_id=row.file_id

        #FildIDごとの解析結果をCSVとして出力
        csv_path=local_save_folder+'csv/'+file_id+'.csv'

        df_content=df_contents[df_contents.file_id==file_id]
        df_content.to_csv(csv_path, index=False)
        
        readed_files += 1
        percentage = (readed_files / total_rows) * 100
        current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        print(f'\r{current_time} 進捗: {percentage:.2f}%', end='')
        
step_message=datetime.now().strftime('%Y-%m-%d %H:%M:%S')+' タグ解析完了'
print('\n'+step_message)
# twillo_message=server_message+"より\n"+step_message
#send_twilio_message(twilio_variables,twillo_message)

2025-01-16 11:03:43 TFIDF解析開始（対象ファイル数1*1組）
2025-01-16 11:03:43 進捗: 100.00%
2025-01-16 11:03:43 タグ解析完了


## 7.Elasticアップロード

In [304]:
#7.1.認証ファイル取得
with open(elastic_password_path, 'r', encoding='utf-8') as f:
    auth=f.readlines()
    dev_auth=auth[5].split(",")
    dev_auth[2]="https://"+dev_auth[2].replace("\n",".es.asia-northeast1.gcp.cloud.es.io:9243")
    dev_mode=tuple(dev_auth)
    prod_auth=auth[9].split(",")
    prod_auth[2]="https://"+prod_auth[2].replace("\n",".es.asia-northeast1.gcp.cloud.es.io:9243")
    prod_mode=tuple(prod_auth)
    
#7.2.Elasticsearchクライアント作成
'''
重要
mode=dev_mode　テスト環境
mode=prod_mode　本番環境
'''
elastic_username,elastic_password,endpoint=prod_mode
es = Elasticsearch(endpoint, basic_auth=(elastic_username, elastic_password))

#7.3.アップロード用データの作成
#進捗表示関連
print(datetime.now().strftime('%Y-%m-%d %H:%M:%S')+' Elasticアップロード開始')
total_rows = len(inputdata[inputdata['テキスト解析'] == 0])
readed_files = 0

for file_index, file_row in inputdata[inputdata['テキスト解析'] == 0].iterrows():
    el_msgs={}
    page_data=pd.read_csv(local_save_folder+'csv/'+file_row.file_id+'.csv',encoding="utf-8")
    
    for page_index, page_row in page_data.iterrows():
        #_ID取得
        elastic_id=str(page_row._id)
        if not elastic_id.endswith('E'):
            elastic_id = str(elastic_id[:9] + 'E' + elastic_id[9:] + 'E')

        current_datetime = datetime.now()
        
        #page_rowからパラメータをコピー（整形）
        page_row['file_id']=str(page_row.file_id)
        page_row['file_page']=int(page_row.file_page)
        
        page_row['tags']=eval(page_row.tags)
        del page_row['_id'],page_row['TF']
        
        #file_rowからパラメータをコピー
        page_row['title']=file_row.title
        page_row['category']=int(file_row.category)
        if file_row.sub_category!="-":
            page_row['sub_category']=int(file_row.sub_category)
        page_row['number_of_pages']=int(file_row.number_of_pages)
        
        page_row['code']=str(file_row.code)
        page_row['affiliation_code']=str(file_row.affiliation_code)
        page_row['organization_code']=str(file_row.organization_code)
        
        page_row['fiscal_year_start']=int(file_row.fiscal_year_start)
        page_row['fiscal_year_end']=int(file_row.fiscal_year_end)
        page_row['source_url']=str(file_row.source_url)
        page_row['hash']=str(file_row.hash)
        
        page_row['collected_at']=file_row.collected_at
        
        #新規パラメータ
        page_row['created_at'],page_row['updated_at']=current_datetime,current_datetime
        page_row['source_url_is_alive']=True
        page_row['published']=1

        #順番調整
        all_possible_indices = ['title','category','sub_category','file_id', 'file_page',
                        'number_of_pages','content_text','code','affiliation_code','organization_code','file_url',
                        'fiscal_year_start', 'fiscal_year_end','source_url','source_url_is_alive',
                        'collected_at','tags', 'created_at', 'updated_at', 'published','hash']
        
        existing_indices = [idx for idx in all_possible_indices if idx in page_row.index]
        page_row = page_row.reindex(existing_indices)
        
        #アップロード先のインディクスを取得
        get_indices=df_config_category[df_config_category[
            'categoryにセットする値（※データ型はint）']==page_row['category']]['インデックス（物理名称）'].values[0]
        
        try:
            uploader = LGDValidator.LocalGovDocumentUploader(elasticsearch_instance=es,
                                                             indices=get_indices,
                                                             id=elastic_id,
                                                             document_data=page_row.to_dict())
            uploader.create_document() # 引数が正しい値・データかを検証し、問題なければ、Elasticsearchにドキュメントデータをアップロードする。問題があれば、エラーとなる。
        except Exception as e:
            # エラー時の処理を記述する。
            el_msgs["Page:"+str(page_row.file_page)]=str(e)
        
    error_pages=len(el_msgs)
    if error_pages==0:
        #すべてのページが問題なくアップロードできた場合
        inputdata.at[file_index, 'Elastic格納'] = 0
        inputdata.at[file_index, 'Elasticエラー詳細'] = "-"
    elif error_pages / len(page_data) >= 0.9:
        #90％以上のページがアップロードできなかった場合
        inputdata.at[file_index, 'Elastic格納'] = 1
        inputdata.at[file_index, 'Elasticエラー詳細'] = list(el_msgs.values())[0]
    else:
        #10％未満のページがアップロードできなかった場合
        inputdata.at[file_index, 'Elastic格納'] = 2
        inputdata.at[file_index, 'Elasticエラー詳細'] = el_msgs
        
    #進捗表示
    readed_files += 1
    percentage = (readed_files / total_rows) * 100
    current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(f'\r{current_time} 進捗: {percentage:.2f}%', end='')
    
step_message=datetime.now().strftime('%Y-%m-%d %H:%M:%S')+' Elasticアップロード完了'
print('\n'+step_message)
twillo_message=server_message+"より\n"+step_message
#send_twilio_message(twilio_variables,twillo_message)
es.close()

#error
inputdata[~inputdata['Elastic格納'].isin(['-',0])]

2024-11-02 22:45:18 Elasticアップロード開始
2024-11-02 22:56:44 進捗: 100.00%
2024-11-02 22:56:44 Elasticアップロード完了


,title,category,category_name,sub_category,sub_category_name,file_id,number_of_pages,code,affiliation_code,affiliation,organization_code,organization,fiscal_year_start,fiscal_year_end,source_url,collected_at,modified_at,submit_id,server_name,重複チェック,URL判定,DL判定,DLエラー詳細,画像化,画像化エラー詳細,テキスト解析,解析エラー詳細,OCR解析,OCRエラー詳細,GCP格納,GCPエラー詳細,画像格納,画像格納エラー詳細,Elastic格納,Elasticエラー詳細,hash


## 8.画像化、画像アップロード

In [305]:
#進捗表示関連
print(datetime.now().strftime('%Y-%m-%d %H:%M:%S')+' 画像化開始')
total_rows = len(inputdata[inputdata['Elastic格納'] == 0])
readed_files = 0

for index, row in inputdata[inputdata['Elastic格納'] == 0].iterrows():   
    file_id=row.file_id
    pdf_path=temp_folder+'pdf/'+file_id+'.pdf'
        
    #画像化(pdf2image)
    preview_dirpath=temp_folder+'preview/'+file_id+'/'
    try:
        convert_to_pic_500(pdf_path, preview_dirpath)
        inputdata.at[index, '画像化'] = 0
        
    except Exception as e:
        inputdata.at[index, '画像化'] = 1
        inputdata.at[index, '画像化エラー詳細'] = str(e)
    
    #進捗表示
    readed_files += 1
    percentage = (readed_files / total_rows) * 100
    current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(f'\r{current_time} 進捗: {percentage:.2f}%', end='')

step_message=datetime.now().strftime('%Y-%m-%d %H:%M:%S')+' 画像化完了'
print('\n'+step_message)
twillo_message=server_message+"より\n"+step_message
#send_twilio_message(twilio_variables,twillo_message)

#5.3.gcp_upload
#5.3.1.google cloud storageのクライアントインスタンスを作成
#Python3からCloud Storageを利用 https://qiita.com/NOGU626/items/88ce6d79b5a22008b004
client_prod = storage.Client.from_service_account_json(service_account_path_prod)
bucket_prod = client_prod.bucket(bucket_name_prod) #バケットのインスタンスを取得

client_backup = storage.Client.from_service_account_json(service_account_path_backup)
bucket_backup = client_backup.bucket(bucket_name_backup) #バケットのインスタンスを取得

#進捗表示関連
print(datetime.now().strftime('%Y-%m-%d %H:%M:%S')+' 画像格納開始')
total_rows = len(inputdata[inputdata['画像化'] == 0])
readed_files = 0

for index, row in inputdata[inputdata['画像化'] == 0].iterrows():   
    file_id=row.file_id
    preview_dirpath=temp_folder+'preview/'+file_id+'/'
    
    #画像アップロード
    try:            
        for pic_name in os.listdir(preview_dirpath):
            #アップロードあとのファイル名を指定(prod)
            upload_as_prod='bunsyo/'+file_id+'/'+pic_name
            blob_prod = bucket_prod.blob(upload_as_prod)

            #アップロードするファイルを指定(prod)
            blob_prod.upload_from_filename(preview_dirpath+pic_name)    
            
        inputdata.at[index, '画像格納'] = 0
            
    except Exception as e:
        #error1.Prodへのアップロードに失敗しました
        inputdata.at[index, '画像格納'] = 1
        inputdata.at[index, '画像格納エラー詳細'] = str(e)
    
    finally:
        # 删除preview_dirpath内的所有文件
        for filename in os.listdir(preview_dirpath):
            file_path = os.path.join(preview_dirpath, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.remove(file_path)
                elif os.path.isdir(file_path):
                    # 如果有子文件夹，这里只能删除空的子文件夹
                    os.rmdir(file_path)
            except Exception as e:
                print(f'Failed to delete {file_path}. Reason: {e}')
        
        # 尝试删除现在应该为空的preview_dirpath文件夹
        try:
            os.rmdir(preview_dirpath)
        except OSError as e:
            print(f"Error: {preview_dirpath} : {e.strerror}")
        
    #進捗表示
    readed_files += 1
    percentage = (readed_files / total_rows) * 100
    current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(f'\r{current_time} 進捗: {percentage:.2f}%', end='')
    
step_message=datetime.now().strftime('%Y-%m-%d %H:%M:%S')+' 画像格納完了'
print('\n'+step_message)
twillo_message=server_message+"より\n"+step_message
#send_twilio_message(twilio_variables,twillo_message)

2024-11-02 22:56:44 画像化開始
2024-11-02 23:29:41 進捗: 100.00%
2024-11-02 23:29:41 画像化完了
2024-11-02 23:29:41 画像格納開始
2024-11-03 01:04:32 進捗: 100.00%
2024-11-03 01:04:32 画像格納完了


## 9. PDF/CSVアップロード

In [306]:
total_rows = len(inputdata[inputdata['DL判定'] == 0])
readed_files = 0

#Python3からCloud Storageを利用 https://qiita.com/NOGU626/items/88ce6d79b5a22008b004
client_prod = storage.Client.from_service_account_json(service_account_path_prod)
bucket_prod = client_prod.bucket(bucket_name_prod) #バケットのインスタンスを取得

client_backup = storage.Client.from_service_account_json(service_account_path_backup)
bucket_backup = client_backup.bucket(bucket_name_backup) #バケットのインスタンスを取得

#進捗表示関連
print(datetime.now().strftime('%Y-%m-%d %H:%M:%S')+' PDFアップロード開始')
for index, row in inputdata[inputdata['DL判定'] == 0].iterrows():
    file_id=row.file_id
    pdf_path=temp_folder+'pdf/'+file_id+'.pdf'
    csv_path=local_save_folder+'csv/'+file_id+'.csv'

    inputdata.at[index, 'GCP格納'] = {}
    inputdata.at[index, 'GCPエラー詳細'] = {}

    #PDFアップロード（Prod）
    try:            
        #アップロードあとのファイル名を指定（Prod）
        upload_as_prod='bunsyo/'+file_id+'.pdf'
        blob_prod = bucket_prod.blob(upload_as_prod)

        #アップロードするファイルを指定（Prod）
        blob_prod.upload_from_filename(pdf_path)

        inputdata.at[index, 'GCP格納']['Prod'] = 0
        inputdata.at[index, 'GCPエラー詳細']['Prod'] = 0

    except Exception as e:
        #アップロードに失敗しました
        inputdata.at[index, 'GCP格納']['Prod'] = 1
        inputdata.at[index, 'GCPエラー詳細']['Prod'] = str(e)

    #進捗表示（Prod）
    readed_files += 0.5
    percentage = (readed_files / total_rows) * 100
    current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(f'\r{current_time} 進捗: {percentage:.2f}%', end='')

    #PDFアップロード（Backup）
    try:            
        #アップロードあとのファイル名を指定（Backup）
        upload_as_backup='bunsyo/pdf/'+file_id+'.pdf'
        blob_backup = bucket_backup.blob(upload_as_backup)

        #アップロードするファイルを指定（Backup）
        blob_backup.upload_from_filename(pdf_path)

        inputdata.at[index, 'GCP格納']['Backup'] = 0
        inputdata.at[index, 'GCPエラー詳細']['Backup'] = 0

    except Exception as e:
        #アップロードに失敗しました
        inputdata.at[index, 'GCP格納']['Backup'] = 1
        inputdata.at[index, 'GCPエラー詳細']['Backup'] = str(e)


    #CSVアップロード（Backup）
    if os.path.exists(csv_path):
        try:            
            #アップロードあとのファイル名を指定（Backup）
            upload_as_backup='bunsyo/csv/'+file_id+'.csv'
            blob_backup = bucket_backup.blob(upload_as_backup)

            #アップロードするファイルを指定（Backup）
            blob_backup.upload_from_filename(csv_path)

            inputdata.at[index, 'GCP格納']['CSV'] = 0
            inputdata.at[index, 'GCPエラー詳細']['CSV'] = 0

        except Exception as e:
            #アップロードに失敗しました
            inputdata.at[index, 'GCP格納']['CSV'] = 1
            inputdata.at[index, 'GCPエラー詳細']['CSV'] = str(e)

    if all(value == 0 for value in inputdata.at[index, 'GCP格納'].values()):
        inputdata.at[index, 'GCP格納'] = 0
        inputdata.at[index, 'GCPエラー詳細'] = "-"

    #進捗表示（Prod）
    readed_files += 0.5
    percentage = (readed_files / total_rows) * 100
    current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(f'\r{current_time} 進捗: {percentage:.2f}%', end='')


#9.1.今回解析結果を出力
report_name=str(inputdata.submit_id.min())+"解析結果レポート.xlsx"
inputdata.set_index("file_id").to_excel(path_teams+path_teams_report_folder+report_name)

#9.3.データフレームの保存先
pickle_path=r"../bunsyo/pickle/"+str(inputdata.submit_id.min())+"_"+inputdata['server_name'].iloc[0]+"_DataFrame.pkl"
inputdata.to_pickle(pickle_path)

#end
error_count='（失敗：'+str(len(inputdata[~(inputdata['GCP格納'].isin([0,'-']))]))+'件）'
step_message=datetime.now().strftime('%Y-%m-%d %H:%M:%S')+' PDFアップロード完了'
print('\n'+'データフレーム格納先：'+pickle_path)
print(step_message)

twillo_message=server_message+"より\n"+step_message
#send_twilio_message(twilio_variables,twillo_message)

inputdata[~(inputdata['GCP格納'].isin([0,'-']))]

2024-11-03 01:04:32 PDFアップロード開始
2024-11-03 01:17:18 進捗: 100.00%
データフレーム格納先：../bunsyo/pickle/202411011_Earth_DataFrame.pkl
2024-11-03 01:17:20 PDFアップロード完了


,title,category,category_name,sub_category,sub_category_name,file_id,number_of_pages,code,affiliation_code,affiliation,organization_code,organization,fiscal_year_start,fiscal_year_end,source_url,collected_at,modified_at,submit_id,server_name,重複チェック,URL判定,DL判定,DLエラー詳細,画像化,画像化エラー詳細,テキスト解析,解析エラー詳細,OCR解析,OCRエラー詳細,GCP格納,GCPエラー詳細,画像格納,画像格納エラー詳細,Elastic格納,Elasticエラー詳細,hash


## 10.解析結果レポート

In [307]:
#10.1.今回解析結果を出力
report_name=str(inputdata.submit_id.min())+"解析結果レポート.xlsx"
inputdata.set_index("file_id").to_excel(path_teams+path_teams_report_folder+report_name)

#10.2.last_submit_idへのデータ追加
columns_to_write = ['file_id', 'source_url', 'category', 'sub_category', 'submit_id', 'server_name']

last_submit_id_file_path =last_submit_id_folder+ inputdata['server_name'].iloc[0]+".csv"
existing_data = pd.read_csv(last_submit_id_file_path)
combined_data = pd.concat([existing_data,inputdata[columns_to_write]],
                          ignore_index=True)

combined_data.drop_duplicates(subset=['file_id'], keep='last', inplace=True)
combined_data.to_csv(last_submit_id_file_path, index=False, columns=columns_to_write)

#10.3.データフレームの保存先
pickle_path=r"../bunsyo/pickle/"+str(inputdata.submit_id.min())+"_"+inputdata['server_name'].iloc[0]+"_DataFrame.pkl"
inputdata.to_pickle(pickle_path)

#end
step_message = f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} {inputdata['submit_id'].min()} {inputdata['server_name'].iloc[0]}: All done."
send_twilio_message(twilio_variables,step_message)
print('データフレーム格納先：'+pickle_path)
print(step_message)

inputdata

データフレーム格納先：../bunsyo/pickle/202411011_Earth_DataFrame.pkl
2024-11-03 01:17:24 202411011 Earth: All done.


,title,category,category_name,sub_category,sub_category_name,file_id,number_of_pages,code,affiliation_code,affiliation,organization_code,organization,fiscal_year_start,fiscal_year_end,source_url,collected_at,modified_at,submit_id,server_name,重複チェック,URL判定,DL判定,DLエラー詳細,画像化,画像化エラー詳細,テキスト解析,解析エラー詳細,OCR解析,OCRエラー詳細,GCP格納,GCPエラー詳細,画像格納,画像格納エラー詳細,Elastic格納,Elasticエラー詳細,hash
0,令和６年度一般会計補正予算（第１号）,4,（自治体）予算,6,（自治体）補正予算,AA0178569,17,462187,46,鹿児島県,2187,霧島市,2024,2024,https://www.city-kirishima.jp/zaisei/documents...,2024-03-30 15:55:00,,202411011,Earth,0,0,0,-,0,-,0,-,-,-,0,-,0,-,0,-,0de6d82659dfe18a05c6d8e3f9103630c9312580028ea8...
1,令和６年度一般会計補正予算説明資料（第１号）,4,（自治体）予算,6,（自治体）補正予算,AA0178570,2,462187,46,鹿児島県,2187,霧島市,2024,2024,https://www.city-kirishima.jp/zaisei/documents...,2024-03-30 15:55:00,,202411011,Earth,0,0,0,-,0,-,0,-,-,-,0,-,0,-,0,-,3508cafe855bb9dbb39bf10fb255539433f86e073be545...
2,令和６年度４月補正予算（専決）,4,（自治体）予算,6,（自治体）補正予算,AA0178571,7,462195,46,鹿児島県,2195,いちき串木野市,2024,2024,https://www.city.ichikikushikino.lg.jp/zaisei1...,2024-03-30 15:55:00,,202411011,Earth,0,0,0,-,0,-,0,-,-,-,0,-,0,-,0,-,20e887d641164b796ac2d85b324b88bee1362e7f2521a8...
3,令６年度 一般会計（第１号）補正予算,4,（自治体）予算,6,（自治体）補正予算,AA0178572,8,462233,46,鹿児島県,2233,南九州市,2024,2024,https://www.city.minamikyushu.lg.jp/material/f...,2024-03-30 15:55:00,,202411011,Earth,0,0,0,-,0,-,0,-,-,-,0,-,0,-,0,-,98853e537d38c8ff79710cdcd13c48da04b3ca12f6a6ed...
4,令和６年度予算 ４月１１日臨時議会 予算説明資料,4,（自治体）予算,6,（自治体）補正予算,AA0178573,2,462233,46,鹿児島県,2233,南九州市,2024,2024,https://www.city.minamikyushu.lg.jp/material/f...,2024-03-30 15:55:00,,202411011,Earth,0,0,0,-,0,-,0,-,-,-,0,-,0,-,0,-,fad28051dd3cfca76faa2e4fb16019c34c3a0daf7b62a5...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1927,令和２年度補正予算（議案第８２号～第８３号）－令和２年９月定例月議会,4,（自治体）予算,6,（自治体）補正予算,AA0180496,44,232297,23,愛知県,2297,豊明市,2020,2020,https://www.city.toyoake.lg.jp/secure/18340/R2...,2024-10-09 11:29:00,,202411011,Earth,0,0,0,-,0,-,0,-,-,-,0,-,0,-,0,-,a16637e927a41fcacaa02b3ccd6c363c879bf01cb273bc...
1928,令和２年度補正予算（議案第５５号・第７４号～第７５号）－令和２年６月定例月議会,4,（自治体）予算,6,（自治体）補正予算,AA0180497,136,232297,23,愛知県,2297,豊明市,2020,2020,https://www.city.toyoake.lg.jp/secure/18097/R2...,2024-10-09 11:31:00,,202411011,Earth,0,0,0,-,0,-,0,-,-,-,0,-,0,-,0,-,0698ddce9919ce25a79eb2e43816561650d0a28a9b61ad...
1929,令和２年度補正予算（議案第３８号～第３９号）－令和２年定例会開会議会,4,（自治体）予算,6,（自治体）補正予算,AA0180498,30,232297,23,愛知県,2297,豊明市,2020,2020,https://www.city.toyoake.lg.jp/secure/17736/R2...,2024-10-09 11:33:00,,202411011,Earth,0,0,0,-,0,-,0,-,-,-,0,-,0,-,0,-,4ad9448d853f70e87c93e556f304356671ad54546dc772...
1930,令和元年度補正予算（議案第２１号～第２９号）－令和２年３月定例月議会,4,（自治体）予算,6,（自治体）補正予算,AA0180499,193,232297,23,愛知県,2297,豊明市,2019,2019,https://www.city.toyoake.lg.jp/secure/16326/R2...,2024-10-09 11:34:00,,202411011,Earth,0,0,0,-,0,-,0,-,-,-,0,-,0,-,0,-,6ab4aa07f9207dae7518c35d4478291483d2609fb7a19f...


In [308]:
###Requirements出力
##!pipreqsnb ./ --encoding=utf8 --force